<a href="https://colab.research.google.com/github/jidemaestri/Financial-Markets/blob/master/03_portfolio_optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Markowitz Frontier

La frontera de Markowitz se calcula para encontrar la combinación óptima de inversiones que ofrece el mejor rendimiento posible con un determinado nivel de riesgo. Esta frontera se basa en el modelo de Markowitz de selección de carteras, que es una forma de análisis de inversiones que toma en cuenta el riesgo y la rentabilidad potencial de diferentes activos financieros. La frontera de Markowitz se representa gráficamente como una curva que muestra la relación entre el rendimiento esperado y el riesgo de una cartera de inversiones. 

In [3]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [5]:
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt


In [166]:
tickers = ["AAPL", "GOOG", "MSFT", "VALE", "MELI", "DIS", "KO", "MCD", "NVDA", "AMD", "PAM", "BBAR", "GGAL", "VIST", "JPM", "BRK", "PEP", "PBR", "BBD", "BABA", "GLOB", "TEO", "TGS", "CEPU", "LOMA", "CRESY"]

In [167]:
data = yf.download(tickers, start="2009-01-01", end="2022-12-31")



[*********************100%***********************]  26 of 26 completed


In [168]:
data["Adj Close"].tail()

,AAPL,AMD,BABA,BBAR,BBD,BRK,CEPU,CRESY,DIS,GGAL,...,MELI,MSFT,NVDA,PAM,PBR,PEP,TEO,TGS,VALE,VIST
Date,,,,,,,,,,,,,,,,,,,,,
2022-12-07,140.940002,70.139999,88.330002,3.01,2.85,NaN,5.29,5.65,92.150002,7.95,...,863.809998,244.369995,161.199997,27.639999,11.08,182.179993,4.56,9.44,16.170000,12.54
2022-12-08,142.649994,70.470001,94.169998,2.81,2.76,NaN,4.98,5.44,92.550003,7.51,...,884.239990,247.399994,171.690002,26.180000,10.78,183.779999,4.61,8.94,16.430000,12.40
2022-12-09,142.160004,68.589996,91.339996,2.77,2.70,NaN,4.93,5.25,93.379997,7.44,...,896.479980,245.419998,170.009995,25.980000,10.71,183.100006,4.45,8.52,16.940001,11.95
2022-12-12,144.490005,70.669998,89.410004,2.88,2.67,NaN,5.33,5.76,94.660004,7.76,...,880.250000,252.509995,175.350006,26.450001,10.33,183.970001,4.60,9.28,16.230000,12.00
2022-12-13,145.470001,71.650002,91.410004,2.85,2.55,NaN,5.21,5.70,94.699997,7.62,...,870.650024,256.920013,180.720001,26.010000,10.14,183.869995,4.34,9.00,16.180000,12.39


In [143]:

# Calcula los retornos de cada acción
returns = data["Adj Close"].pct_change().dropna()



In [144]:
returns.tail()

,AAPL,AMD,BABA,BBAR,BRK,CEPU,CRESY,DIS,DPZ,GGAL,...,MCD,MELI,MSFT,NVDA,PAM,PEP,TEO,TGS,VALE,VIST
Date,,,,,,,,,,,,,,,,,,,,,
2022-12-07,-0.013785,-0.001850,-0.034117,0.045139,0.0,0.058000,0.046296,-0.001517,0.007527,0.041940,...,-0.005262,0.000834,-0.003060,0.008319,0.022189,0.003028,0.031674,0.033954,-0.035203,-0.016471
2022-12-08,0.012133,0.004705,0.066116,-0.066445,0.0,-0.058601,-0.037168,0.004341,0.007524,-0.055346,...,0.011282,0.023651,0.012399,0.065074,-0.052822,0.008783,0.010965,-0.052966,0.016079,-0.011164
2022-12-09,-0.003435,-0.026678,-0.030052,-0.014235,0.0,-0.010040,-0.034926,0.008968,-0.032779,-0.009321,...,-0.004938,0.013842,-0.008003,-0.009785,-0.007639,-0.003700,-0.034707,-0.046980,0.031041,-0.036290
2022-12-12,0.016390,0.030325,-0.021130,0.039711,0.0,0.081136,0.097143,0.013708,0.001930,0.043011,...,0.016836,-0.018104,0.028889,0.031410,0.018091,0.004751,0.033708,0.089202,-0.041913,0.004184
2022-12-13,0.006782,0.013867,0.022369,-0.010417,0.0,-0.022514,-0.010417,0.000422,-0.004073,-0.018041,...,-0.008459,-0.010906,0.017465,0.030624,-0.016635,-0.000544,-0.056522,-0.030172,-0.003081,0.032500


In [145]:
returns_pct = returns.mul(100)


In [146]:
returns_pct.tail()

,AAPL,AMD,BABA,BBAR,BRK,CEPU,CRESY,DIS,DPZ,GGAL,...,MCD,MELI,MSFT,NVDA,PAM,PEP,TEO,TGS,VALE,VIST
Date,,,,,,,,,,,,,,,,,,,,,
2022-12-07,-1.378491,-0.184997,-3.411695,4.513884,0.0,5.799999,4.629630,-0.151695,0.752661,4.193967,...,-0.526178,0.083418,-0.305973,0.831927,2.218929,0.302807,3.167418,3.395394,-3.520287,-1.647059
2022-12-08,1.213276,0.470490,6.611566,-6.644520,0.0,-5.860112,-3.716815,0.434077,0.752402,-5.534586,...,1.128216,2.365103,1.239923,6.507448,-5.282197,0.878256,1.096495,-5.296610,1.607917,-1.116430
2022-12-09,-0.343491,-2.667809,-3.005205,-1.423486,0.0,-1.004020,-3.492648,0.896806,-3.277860,-0.932093,...,-0.493802,1.384238,-0.800322,-0.978512,-0.763945,-0.370004,-3.470723,-4.697977,3.104079,-3.629031
2022-12-12,1.639000,3.032515,-2.112977,3.971124,0.0,8.113592,9.714290,1.370750,0.193027,4.301078,...,1.683571,-1.810412,2.888924,3.140999,1.809089,0.475148,3.370789,8.920179,-4.191269,0.418412
2022-12-13,0.678245,1.386732,2.236886,-1.041674,0.0,-2.251405,-1.041674,0.042249,-0.407324,-1.804128,...,-0.845925,-1.090596,1.746473,3.062444,-1.663518,-0.054360,-5.652169,-3.017239,-0.308067,3.250003


In [147]:
# Calcula la matriz de covarianzas de la cartera
cov_matrix = np.cov(returns_pct)


In [90]:
cov_matrix

array([[10.11585574,  2.16576382, -6.57189884, ...,  2.84264702,
        -5.40891697, -0.84122502],
       [ 2.16576382, 10.16248376, -6.94508429, ...,  2.45594866,
        -2.90459381,  1.36800941],
       [-6.57189884, -6.94508429,  9.77850831, ..., -3.1630933 ,
         4.07781568, -0.16726995],
       ...,
       [ 2.84264702,  2.45594866, -3.1630933 , ...,  2.02841758,
        -2.93881815, -1.18301326],
       [-5.40891697, -2.90459381,  4.07781568, ..., -2.93881815,
         5.45946098,  1.85031614],
       [-0.84122502,  1.36800941, -0.16726995, ..., -1.18301326,
         1.85031614,  2.14889085]])

In [91]:
cov_matrix_df = pd.DataFrame(cov_matrix)

In [148]:
# Crear una lista de posibles rendimientos esperados
expected_returns = np.linspace(returns_pct.min(), returns_pct.max(), 100)


In [149]:
expected_returns_df = pd.DataFrame(expected_returns)


In [150]:
expected_returns_df.tail()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
95,10.976977,15.243394,34.739450,17.834098,0.0,17.967582,36.652262,13.298153,24.179641,19.588958,...,16.751679,18.162541,13.046953,15.717684,19.102708,11.952155,12.250538,13.333638,19.977975,27.492271
96,11.227942,15.557945,35.245573,18.609726,0.0,18.745295,37.439027,13.576693,24.534929,20.385867,...,17.095121,18.531398,13.339437,16.077365,19.870268,12.198266,12.730815,13.979628,20.360579,28.119201
97,11.478907,15.872496,35.751695,19.385355,0.0,19.523008,38.225792,13.855233,24.890217,21.182776,...,17.438564,18.900255,13.631922,16.437047,20.637828,12.444376,13.211092,14.625619,20.743182,28.746132
98,11.729872,16.187047,36.257818,20.160983,0.0,20.300722,39.012558,14.133774,25.245504,21.979684,...,17.782006,19.269112,13.924406,16.796729,21.405388,12.690486,13.691369,15.271610,21.125786,29.373062
99,11.980837,16.501598,36.763940,20.936612,0.0,21.078435,39.799323,14.412314,25.600792,22.776593,...,18.125449,19.637969,14.216890,17.156411,22.172948,12.936597,14.171646,15.917601,21.508389,29.999992


In [70]:

# Crear una lista de posibles niveles de volatilidad
volatilities = []


In [151]:
n = len(data["Adj Close"].columns)

# Crear un array de n elementos
weights = np.ones(n)
weights = weights / sum(weights)


In [152]:
weights


array([0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04,
       0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04,
       0.04, 0.04, 0.04])

### Portfolio Optimizer

In [154]:
!pip install PyPortfolioOpt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [169]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns



In [170]:
portfolio = data["Adj Close"]

In [171]:

mu = expected_returns.mean_historical_return(portfolio)
S = risk_models.sample_cov(portfolio)



/usr/local/lib/python3.8/dist-packages/pypfopt/risk_models.py:68: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.warn(


In [172]:
mu

AAPL     0.329091
AMD      0.276816
BABA    -0.003249
BBAR     0.023754
BBD     -0.030168
BRK     -0.198114
CEPU    -0.213677
CRESY   -0.012435
DIS      0.115715
GGAL     0.094788
GLOB     0.397033
GOOG     0.195081
JPM      0.137170
KO       0.110601
LOMA    -0.214043
MCD      0.142948
MELI     0.326254
MSFT     0.224998
NVDA     0.381701
PAM      0.057195
PBR     -0.004182
PEP      0.121747
TEO     -0.004831
TGS      0.173810
VALE     0.065283
VIST     0.095459
dtype: float64

In [173]:
S

,AAPL,AMD,BABA,BBAR,BBD,BRK,CEPU,CRESY,DIS,GGAL,...,MELI,MSFT,NVDA,PAM,PBR,PEP,TEO,TGS,VALE,VIST
AAPL,0.084654,0.068348,0.047588,0.042443,0.041607,3.883429e-01,0.051155,0.033638,0.034519,0.043965,...,0.061122,0.045172,0.067793,0.030767,0.047047,0.020434,0.023926,0.025615,0.049002,7.549279e-02
AMD,0.068348,0.342217,0.077670,0.055276,0.067481,9.610440e-01,0.068549,0.049038,0.053903,0.059396,...,0.106688,0.064483,0.146284,0.035823,0.083044,0.025141,0.038135,0.037671,0.088941,9.529119e-02
BABA,0.047588,0.077670,0.175929,0.045831,0.047027,3.262279e-01,0.052167,0.038053,0.033082,0.044354,...,0.090030,0.046182,0.079455,0.036229,0.050184,0.012610,0.026447,0.033587,0.063939,7.160391e-02
BBAR,0.042443,0.055276,0.045831,0.336643,0.099234,-1.352410e-01,0.305033,0.138557,0.047015,0.256647,...,0.078291,0.041985,0.057976,0.185625,0.106085,0.021085,0.127632,0.155326,0.083580,2.343605e-01
BBD,0.041607,0.067481,0.047027,0.099234,0.192760,-2.847818e-01,0.112745,0.070810,0.050847,0.095339,...,0.072802,0.042354,0.058602,0.065845,0.164185,0.026100,0.059691,0.058527,0.124766,1.447379e-01
BRK,0.388343,0.961044,0.326228,-0.135241,-0.284782,2.947748e+03,-0.000188,0.242682,0.218465,-0.089462,...,0.372441,0.140900,0.839570,0.094630,-0.151402,0.070831,-0.168535,-0.300807,-0.150943,-3.928763e-07
CEPU,0.051155,0.068549,0.052167,0.305033,0.112745,-1.877193e-04,0.444770,0.199128,0.052794,0.300827,...,0.084585,0.050148,0.069837,0.259109,0.124103,0.025461,0.160716,0.240729,0.096994,2.273520e-01
CRESY,0.033638,0.049038,0.038053,0.138557,0.070810,2.426819e-01,0.199128,0.233812,0.038482,0.136996,...,0.058266,0.032013,0.049343,0.117570,0.080317,0.017599,0.085055,0.110832,0.070089,1.916377e-01
DIS,0.034519,0.053903,0.033082,0.047015,0.050847,2.184654e-01,0.052794,0.038482,0.075082,0.049701,...,0.057656,0.036142,0.051986,0.030312,0.054546,0.020593,0.032383,0.029072,0.050019,9.032264e-02
GGAL,0.043965,0.059396,0.044354,0.256647,0.095339,-8.946165e-02,0.300827,0.136996,0.049701,0.321388,...,0.075040,0.042608,0.060520,0.185091,0.103730,0.021187,0.129753,0.158055,0.080177,2.348076e-01


In [174]:

# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()

In [176]:

cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("weights.csv")  # saves to file
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('AAPL', 0.53394), ('AMD', 0.0), ('BABA', 0.0), ('BBAR', 0.0), ('BBD', 0.0), ('BRK', 0.0), ('CEPU', 0.0), ('CRESY', 0.0), ('DIS', 0.0), ('GGAL', 0.0), ('GLOB', 0.16017), ('GOOG', 0.0), ('JPM', 0.0), ('KO', 0.0), ('LOMA', 0.0), ('MCD', 0.1459), ('MELI', 0.0), ('MSFT', 0.0), ('NVDA', 0.08826), ('PAM', 0.0), ('PBR', 0.0), ('PEP', 0.0525), ('TEO', 0.0), ('TGS', 0.01921), ('VALE', 0.0), ('VIST', 0.0)])
Expected annual return: 30.4%
Annual volatility: 24.5%
Sharpe Ratio: 1.16


(0.3035887753644613, 0.2447604385145533, 1.1586381242228379)

In [177]:
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices



In [178]:

latest_prices = get_latest_prices(portfolio)


In [179]:

da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=15000)
allocation, leftover = da.greedy_portfolio()


In [180]:
print("Total Stocks: ", allocation)
print("Funds remaining: ${:.2f}".format(leftover))

Total Stocks:  {'AAPL': 55, 'GLOB': 13, 'MCD': 8, 'NVDA': 7, 'PEP': 4, 'TGS': 32}
Funds remaining: $90.07
